# 10 -1 간단한 함수 만들기

In [1]:
def my_sq(x):
    return x**2

In [2]:
def my_exp(x,n):
    return x **n

In [3]:
my_sq(4)

16

In [4]:
my_exp(2,4)

16

# 10-2 apply 메소드 사용하기 -기초 

In [5]:
import pandas as pd

df = pd.DataFrame({'a':[10,20,30], 'b':[20,30,40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [6]:
print(df['a']**2)

0    100
1    400
2    900
Name: a, dtype: int64


In [7]:
# apply메서드에 전달하는 함수(my_sq)가 1개의 인자를 받도록 구성되어있으면 인잣값 생략해야한다.
# sq 변수에 담았다.
sq = df['a'].apply(my_sq)
sq

0    100
1    400
2    900
Name: a, dtype: int64

In [8]:
# apply메서드에 전달하는 함수(my_exp)가 2개의 인자를 받도록 구성됨.
# ex 변수에 담았다.
ex = df['a'].apply(my_exp, n=3)
ex

0     1000
1     8000
2    27000
Name: a, dtype: int64

In [9]:
def print_me(x):
    print(x)

In [10]:
#데이터 프레임에 함수를 적용할 때는 열방향일지, 행방향일지 정해야 함.
# axis =0 => 함수를 열 방향으로 지정 (지정안하면 열방향이 기본값)
df.apply(print_me, axis=0)

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


a    None
b    None
dtype: object

In [11]:
print(df['a'])
print(df['b'])

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


In [12]:
def avg_3(x,y,z):
    return(x+y+z)/3

In [13]:
# 인자값이 3개가 필요한데 1개 밖에 못 받았다는 오료 발생
df.apply(avg_3)

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [14]:
def avg_3_apply(col):
    x=col[0]
    y=col[1]
    z=col[2]
    return (x+y+z) /3

In [15]:
print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [16]:
def avg_3_apply(col):
    sum =0
    for item in col:
        sum += item
    return sum / df.shape[0]

In [17]:
def avg_3_apply(col):
    sum =0
    for item in col:
        sum += item
    return sum / df.shape[0]

In [18]:
def avg_2_apply(row):
    sum=0
    for item in row:
        sum += item
    return sum/ df.shape[1]

In [19]:
#행방향으로 데이터프레임에 함수 적용하기
df.apply(avg_2_apply, axis =1)

0    15.0
1    25.0
2    35.0
dtype: float64

# 10-3 apply 메서드 사용하기 -고급

In [20]:
import seaborn as sns
titanic = sns.load_dataset("titanic")

In [21]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [22]:
#누락값 개수 반환하는 함수 => count_missing함수
import numpy as np
def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [23]:
cmis_col= titanic.apply(count_missing)
cmis_col

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [24]:
#누락값으로 비율로 계산하는 함수 =>  prop_missing
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size #데이터 프레임의 전체 데이터 수 확인
    return num / dem

In [25]:
pmis_col = titanic.apply(prop_missing)
pmis_col

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64

In [26]:
# 누락값이 아닌 데이터의 비율 => prop_complete
def prop_complete(vec):
    return 1- prop_missing(vec)

In [27]:
# 데이터 프레임 누락값 => 행으로 처리하기
cmis_row =  titanic.apply(count_missing, axis =1)
pmis_row =  titanic.apply(prop_missing, axis =1)
pcom_row =  titanic.apply(prop_complete, axis =1)

In [28]:
print(cmis_row.head())
print(pmis_row.head())
print(pcom_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64
0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64
0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [29]:
#누락값을 구하여 데이터 프레임에 추가하기
titanic['num_missing'] = titanic.apply(count_missing, axis =1)
titanic.head

<bound method NDFrame.head of      survived  pclass     sex   age  sibsp  parch      fare embarked   class  \
0           0       3    male  22.0      1      0    7.2500        S   Third   
1           1       1  female  38.0      1      0   71.2833        C   First   
2           1       3  female  26.0      0      0    7.9250        S   Third   
3           1       1  female  35.0      1      0   53.1000        S   First   
4           0       3    male  35.0      0      0    8.0500        S   Third   
5           0       3    male   NaN      0      0    8.4583        Q   Third   
6           0       1    male  54.0      0      0   51.8625        S   First   
7           0       3    male   2.0      3      1   21.0750        S   Third   
8           1       3  female  27.0      0      2   11.1333        S   Third   
9           1       2  female  14.0      1      0   30.0708        C  Second   
10          1       3  female   4.0      1      1   16.7000        S   Third   
11        

In [30]:
# 누락값이 있는 데이터만 따로 모아서 보기
# 누락값 2개 이상인 데이터 추출
titanic.loc[titanic.num_missing>1,:].sample(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,num_missing
186,1,3,female,NaN,1,0,15.5000,Q,Third,woman,False,NaN,Queenstown,yes,False,2
517,0,3,male,NaN,0,0,24.1500,Q,Third,man,True,NaN,Queenstown,no,True,2
589,0,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,2
552,0,3,male,NaN,0,0,7.8292,Q,Third,man,True,NaN,Queenstown,no,True,2
47,1,3,female,NaN,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,yes,True,2
121,0,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,2
612,1,3,female,NaN,1,0,15.5000,Q,Third,woman,False,NaN,Queenstown,yes,False,2
692,1,3,male,NaN,0,0,56.4958,S,Third,man,True,NaN,Southampton,yes,True,2
61,1,1,female,38.0,0,0,80.0000,NaN,First,woman,False,B,NaN,yes,True,2
347,1,3,female,NaN,1,0,16.1000,S,Third,woman,False,NaN,Southampton,yes,False,2
